In [2]:
import re
import os
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import seaborn as sns
import numpy as np
import librosa
import os
import librosa.display
import random
from sklearn import preprocessing

In [3]:
files = []
for file in sorted(os.listdir(r"/iiscleap-Coswara-Data-bf300ae/compiled")):
    if file.endswith(".wav"):
        files.append(file)
len(files)

859

In [4]:
meta = pd.DataFrame()
meta['filename'] = files
meta

,filename
0,00xKcQMmcAhX8CODgBBLOe7Dm0T2.wav
1,05acPS4aRGfvuOfku11Za8zve8i2.wav
2,05bieNLXPuaIWEVaX81EkbbjVrh1.wav
3,0HIgO2EhOOW1msCbEw1kC8Qsx6D3.wav
4,0Ha52POVIxTKEPqI1eGpIoMHUd52.wav
...,...
854,zir8PPnBwfOaT05GrKj9nK4opfk1.wav
855,zjVy4fN2ICMeFuBCFztx80kVtU32.wav
856,zpjoDhhHvJcfhSHTK9i895kmr2f1.wav
857,zt2ssuTmZESDhYPW40dyo5RW2sT2.wav


In [14]:
audio_lengths = []

def get_audio_length(audio_file_path):
    audio_data, sample_rate = librosa.load(audio_file_path, sr=22050)
    audio_length = len(audio_data) / sample_rate
    return audio_length

for i, f in enumerate(meta['filename'].tolist()):
    length = get_audio_length(r"iiscleap-Coswara-Data-bf300ae/compiled/" + f)
    audio_lengths.append(length)

print(len(audio_lengths))

859


In [23]:
meta['duration'] = audio_lengths
meta.drop(meta[meta["duration"] < 5].index, inplace=True)
meta

,filename,duration
0,00xKcQMmcAhX8CODgBBLOe7Dm0T2.wav,19.968027
1,05acPS4aRGfvuOfku11Za8zve8i2.wav,11.264036
2,05bieNLXPuaIWEVaX81EkbbjVrh1.wav,14.165351
3,0HIgO2EhOOW1msCbEw1kC8Qsx6D3.wav,19.504762
4,0Ha52POVIxTKEPqI1eGpIoMHUd52.wav,21.930703
...,...,...
854,zir8PPnBwfOaT05GrKj9nK4opfk1.wav,16.997007
855,zjVy4fN2ICMeFuBCFztx80kVtU32.wav,11.434694
856,zpjoDhhHvJcfhSHTK9i895kmr2f1.wav,13.141361
857,zt2ssuTmZESDhYPW40dyo5RW2sT2.wav,22.101361


In [25]:
np.average(audio_lengths)

15.50785557218619

In [12]:
del meta['duration']
meta

,filename
0,00xKcQMmcAhX8CODgBBLOe7Dm0T2.wav
1,05acPS4aRGfvuOfku11Za8zve8i2.wav
2,05bieNLXPuaIWEVaX81EkbbjVrh1.wav
3,0HIgO2EhOOW1msCbEw1kC8Qsx6D3.wav
4,0Ha52POVIxTKEPqI1eGpIoMHUd52.wav
...,...
854,zir8PPnBwfOaT05GrKj9nK4opfk1.wav
855,zjVy4fN2ICMeFuBCFztx80kVtU32.wav
856,zpjoDhhHvJcfhSHTK9i895kmr2f1.wav
857,zt2ssuTmZESDhYPW40dyo5RW2sT2.wav


In [26]:
def process_audio(audio_path, target_duration=15):
    audio_data, sample_rate = librosa.load(r"iiscleap-Coswara-Data-bf300ae/compiled/" + audio_path, sr=22050)
    duration = len(audio_data) / sample_rate

    # Pad if audio is shorter than 25s
    if duration < target_duration:
        # Normalise
        max_abs = np.max(np.abs(audio_data))
        normalized_audio = audio_data / max_abs

        # Extracting length of padding needed
        target_samples = int(target_duration * sample_rate)
        current_samples = len(normalized_audio)
        total_pad_samples = max(0, target_samples - current_samples)

        # Splitting the needed padding length into two, randomly
        pad_samples_begin = random.randint(1, total_pad_samples - 1)
        pad_samples_end = total_pad_samples - pad_samples_begin

        silence_begin = np.zeros(pad_samples_begin, dtype=np.float32)
        silence_end = np.zeros(pad_samples_end, dtype=np.float32)
        
        padded_audio = np.concatenate((silence_begin, normalized_audio, silence_end))
    
    # Truncate audio if greater than 25s
    elif duration > target_duration:
        start_time = random.uniform(0, duration - target_duration)
        end_time = start_time + target_duration
        padded_audio = audio_data[int(start_time * sample_rate):int(end_time * sample_rate)]
    
    sf.write(f'processed_audio/{audio_path}', padded_audio, 22050)


In [29]:
for i, f in enumerate(meta["filename"]):
    process_audio(f)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


C:\Users\javin\AppData\Local\Temp\ipykernel_39544\3913237367.py:9: RuntimeWarning: invalid value encountered in divide
  normalized_audio = audio_data / max_abs


24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292


In [31]:
def extract_features(audio_file_list):
    feature_list = []
    for i, audio_file in enumerate(audio_file_list):
        print(i)
        # Load audio file
        y, sr = librosa.load(("processed_audio/"+audio_file), sr=22050)
        print("Sampling rate: ", sr)

        # Extract features
        mfccs = librosa.feature.mfcc(y=y, sr=sr).mean(axis=1)
        zcr = librosa.feature.zero_crossing_rate(y=y).mean(axis=1)
        rms = librosa.feature.rms(y=y).mean(axis=1)
        spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean(axis=1)
        entropy = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(axis=1)
        autocorr = librosa.autocorrelate(y=y).mean()
        
        # Create a dictionary to store features
        features = {
            'filename': [audio_file],
            'mfccs': mfccs.tolist(),
            'zcr': zcr.tolist(),
            'rms': rms.tolist(),
            'spectral_centroid': spec_centroid.tolist(),
            'entropy': entropy.tolist(),
            'autocorr': [autocorr]
        }

        feature_list.append(features)

    return feature_list

def save_to_dataframe(features_list):
    final_features = []
    for i, features in enumerate(features_list):
        feature_set = features['filename'] + features['mfccs'] + features['zcr'] + features['rms'] + features['spectral_centroid'] + features['entropy'] + features['autocorr']

        final_features.append(feature_set)

    mfcc_labels = [f'mfcc_{i}' for i in range(1, len(features_list[0]['mfccs'])+1)]
    
    cols = ['filename'] + mfcc_labels + ["zcr", "rms", "spectral_centroid", "entropy", "autocorr"]

    df = pd.DataFrame(final_features, columns=cols)
    return df

extracted_features = extract_features(meta["filename"])
features_df = save_to_dataframe(extracted_features)

# Display the DataFrame
features_df.to_csv("DL_features.csv", index=False)
features_df

0
Sampling rate:  22050
1
Sampling rate:  22050
2
Sampling rate:  22050
3
Sampling rate:  22050
4
Sampling rate:  22050
5
Sampling rate:  22050
6
Sampling rate:  22050
7
Sampling rate:  22050
8
Sampling rate:  22050
9
Sampling rate:  22050
10
Sampling rate:  22050
11
Sampling rate:  22050
12
Sampling rate:  22050
13
Sampling rate:  22050
14
Sampling rate:  22050
15
Sampling rate:  22050
16
Sampling rate:  22050
17
Sampling rate:  22050
18
Sampling rate:  22050
19
Sampling rate:  22050
20
Sampling rate:  22050
21
Sampling rate:  22050
22
Sampling rate:  22050
23
Sampling rate:  22050
24
Sampling rate:  22050
25
Sampling rate:  22050
26
Sampling rate:  22050
27
Sampling rate:  22050
28
Sampling rate:  22050
29
Sampling rate:  22050
30
Sampling rate:  22050
31
Sampling rate:  22050
32
Sampling rate:  22050
33
Sampling rate:  22050
34
Sampling rate:  22050
35
Sampling rate:  22050
36
Sampling rate:  22050
37
Sampling rate:  22050
38
Sampling rate:  22050
39
Sampling rate:  22050
40
Samplin

,filename,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,zcr,rms,spectral_centroid,entropy,autocorr
0,00xKcQMmcAhX8CODgBBLOe7Dm0T2.wav,-447.743805,103.828560,-21.841188,52.653664,-27.412926,27.014709,-11.129892,-1.274972,-4.301982,...,-8.428254,-4.225214,-1.307319,-8.105540,3.772165,0.166558,0.011288,2127.288757,1905.862672,0.000171
1,05acPS4aRGfvuOfku11Za8zve8i2.wav,-489.946899,-14.360781,-21.813292,28.158346,-32.506714,32.679108,-25.164295,6.734664,-10.159193,...,-3.997098,-2.612487,-6.774116,4.888609,-2.243696,0.288181,0.030725,3193.392547,1328.811221,0.001770
2,05bieNLXPuaIWEVaX81EkbbjVrh1.wav,-307.034454,36.075726,-89.717468,46.915504,-44.890430,10.575853,-39.940327,2.515210,-21.651455,...,-5.087392,5.297164,-10.263174,7.065706,-3.341948,0.258652,0.077348,2854.927590,1643.958392,0.016926
3,0HIgO2EhOOW1msCbEw1kC8Qsx6D3.wav,-751.703247,16.378922,-11.127428,6.554915,-5.165810,14.712893,-7.283758,-11.102134,-5.422174,...,-6.244010,0.572256,-6.682385,-1.631146,10.075688,0.525748,0.000221,4487.396006,2971.554041,0.000039
4,0Ha52POVIxTKEPqI1eGpIoMHUd52.wav,-429.340454,117.565086,-23.568779,28.609085,-21.460163,12.717691,-14.573008,-3.282129,-4.773358,...,-9.318371,-7.732528,-3.934705,-8.185739,-5.895720,0.094375,0.011088,1669.188375,1817.910193,0.000240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,zir8PPnBwfOaT05GrKj9nK4opfk1.wav,-582.404419,114.389679,-14.094843,-3.666786,-1.280706,14.234558,10.749000,-17.872663,-8.637360,...,-2.927632,1.318294,-1.232313,-7.955056,0.225979,0.115506,0.001685,1993.613465,2148.747085,0.000160
810,zjVy4fN2ICMeFuBCFztx80kVtU32.wav,-552.880615,47.598206,27.703812,16.483541,9.671890,5.618227,2.387922,0.827176,0.722942,...,-2.294649,-2.581784,-3.250887,-3.375744,-2.995355,0.152231,0.013261,1314.077122,1485.606247,0.000542
811,zpjoDhhHvJcfhSHTK9i895kmr2f1.wav,-457.989258,3.744323,-62.524986,31.909927,-31.627827,10.098919,-24.918652,-7.770853,-14.418571,...,-3.291226,3.451354,-1.629987,-5.474532,0.859572,0.166408,0.040136,2086.392367,977.134798,0.002869
812,zt2ssuTmZESDhYPW40dyo5RW2sT2.wav,-677.757080,18.673986,-26.970728,34.606808,-26.854483,26.132547,9.009939,14.525517,1.158805,...,1.468389,2.198663,-9.581820,-1.810256,1.247827,0.337880,0.001529,3574.904632,2315.484572,0.000038


In [32]:
features_df_readfromfile = pd.read_csv('DL_features.csv')
filenames = features_df_readfromfile['filename'].tolist()
del features_df_readfromfile['filename']

x = features_df_readfromfile.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
features_df2 = pd.DataFrame(x_scaled)
features_df2.columns = features_df_readfromfile.columns

normalised_meta = pd.DataFrame(filenames, columns=['filename'])
normalised_meta = pd.concat([normalised_meta, features_df2], axis=1)

normalised_meta

,filename,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,zcr,rms,spectral_centroid,entropy,autocorr
0,00xKcQMmcAhX8CODgBBLOe7Dm0T2.wav,0.690421,0.778638,0.592189,0.723691,0.504307,0.728239,0.595897,0.492698,0.521904,...,0.457797,0.500294,0.472224,0.460821,0.648416,0.215184,0.011578,0.338502,0.488048,1.091996e-09
1,05acPS4aRGfvuOfku11Za8zve8i2.wav,0.647799,0.180692,0.592365,0.555505,0.461027,0.784429,0.410131,0.628235,0.433516,...,0.584191,0.543234,0.319321,0.820473,0.422211,0.372315,0.031513,0.508144,0.340278,1.131167e-08
2,05bieNLXPuaIWEVaX81EkbbjVrh1.wav,0.832529,0.435861,0.164342,0.684293,0.355809,0.565167,0.214549,0.556834,0.260094,...,0.553091,0.753838,0.221734,0.880731,0.380915,0.334165,0.079333,0.454286,0.420980,1.081505e-07
3,0HIgO2EhOOW1msCbEw1kC8Qsx6D3.wav,0.383441,0.336211,0.659721,0.407175,0.693330,0.606206,0.646807,0.326404,0.504999,...,0.520100,0.628032,0.321887,0.640019,0.885437,0.679239,0.000227,0.714050,0.760947,2.480386e-10
4,0Ha52POVIxTKEPqI1eGpIoMHUd52.wav,0.709007,0.848134,0.581299,0.558600,0.554885,0.586414,0.550323,0.458733,0.514790,...,0.432407,0.406907,0.398738,0.458601,0.284890,0.121928,0.011373,0.265607,0.465525,1.534657e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,zir8PPnBwfOaT05GrKj9nK4opfk1.wav,0.554422,0.832069,0.641017,0.336993,0.726340,0.601461,0.885497,0.211835,0.456481,...,0.614696,0.647896,0.474322,0.464986,0.515074,0.149227,0.001728,0.317231,0.550245,1.023273e-09
810,zjVy4fN2ICMeFuBCFztx80kVtU32.wav,0.584240,0.494156,0.904488,0.475346,0.819400,0.515987,0.774826,0.528270,0.597731,...,0.632751,0.544052,0.417864,0.591732,0.393947,0.196675,0.013602,0.209101,0.380430,3.465830e-09
811,zpjoDhhHvJcfhSHTK9i895kmr2f1.wav,0.680074,0.272289,0.335745,0.581264,0.468495,0.560436,0.413383,0.382776,0.369241,...,0.604325,0.704691,0.463199,0.533642,0.538898,0.214990,0.041167,0.331994,0.250222,1.833285e-08
812,zt2ssuTmZESDhYPW40dyo5RW2sT2.wav,0.458122,0.347822,0.559856,0.599781,0.509052,0.719488,0.862478,0.760070,0.604309,...,0.740087,0.671337,0.240791,0.635062,0.553497,0.436523,0.001569,0.568851,0.592942,2.438315e-10


In [36]:
metadata = pd.read_csv(r"iiscleap-Coswara-Data-bf300ae/compiled/data_description.csv")

In [37]:
metadata

,filename,diagnosis
0,1irqEfgfrbXwr2ymwtkMB1Y0rnB3.wav,healthy
1,2jFheyh8bLO5peh0HPYGYjH5N9x2.wav,healthy
2,GANtY5XDqOUFMC2lEizSdBuainB2.wav,no_resp_illness_exposed
3,gfAVpM1IV5RhiO6CAiOjjn1AHVG2.wav,healthy
4,GMjhTrVkYsfH4RWjGFLCskzNnys2.wav,healthy
...,...,...
854,rlklHwPKGycGalH7q9XMRnUqXxd2.wav,healthy
855,RV4V6j3c5cbSb2B4pY9vlF6kbeE2.wav,healthy
856,Sy1VK1UgX0ZMcMJitgPal1sp8jj2.wav,healthy
857,vX3NZt9tyQUhXgS4dlz55VGEMdU2.wav,healthy


In [39]:
normal_with_diag = normalised_meta.merge(metadata)
normal_with_diag.to_csv("normalised_features_with_diagnosis.csv", index=False)